# Adding Interactivity

## Objectives

- Demonstrate various interactive features in Bokeh, including hover tools and linked brushing.
- Showcase different data visualizations like pie charts, lines, and scatter plots.
- Illustrate the interaction between visual elements across multiple charts.
- Apply interactive tools to enhance user engagement and data exploration.

## Background

This notebook uses the Bokeh library to implement interactivity in data visualizations. By leveraging features like the Hover Tool, linked brushing, and dynamic layouts, the notebook enhances the user's ability to engage with and understand complex datasets. The examples use real-world data from different domains to showcase Bokeh's versatility in creating interactive, informative visualizations.

## Datasets Used

- **Sunspots Dataset**: Contains historical data on monthly mean total sunspot numbers. It helps visualize solar activity trends over time.
- **Automobile Dataset from UCI**: It includes various attributes of automobiles like price, mpg, and horsepower, which are used to explore relationships between these variables through interactive scatter plots.

## Hover Tool

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 8)

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.transform import cumsum
output_notebook()

Loading BokehJS ...

The Hover Tool in Bokeh displays more information in a popup whenever the user hovers over a glyph.

In [2]:
from bokeh.models import HoverTool

### Plotting lines and points

The data

In [3]:
x1 = [1, 2, 3, 4, 5, 6]
y1 = [3, 4, 2, 3, 6, 5]

In [4]:
# Hover over data points to see the values
p = figure(title="Hover over Points", x_axis_label='x', y_axis_label='y',  y_range=(0, 7),    
    tools=[HoverTool()], tooltips="(@x,@y)", width=600, height=400)
p.line(x1, y1, line_width=2)
p.scatter(x1, y1, marker='circle', size=15, line_color='white')
show(p);

### Pie Chart

In [5]:
from math import pi

In [6]:
fruits = ['Guava','Mango','Orange','Lime']
counts = [2, 4, 5, 3]
angles = [counts[i]/np.sum(counts) * 2*pi for i in range(len(counts))]
colors = ['lightcoral', 'gold', 'darkorange', 'limegreen']    

In [7]:
data={
    'fruits': fruits,
    'counts': counts,    
    'angles': angles,
    'colors': colors
}

In [8]:
source_fruit = ColumnDataSource(data = data)

In [9]:
# Mouse over graph to visualize values
p2 = figure(tools=[HoverTool()], tooltips="(@fruits,@counts)",  
    width=500, height=500) 
p2.axis.visible = False
p2.grid.grid_line_color = None
p2.wedge(x=2, y=0, radius=0.8,
        start_angle=cumsum('angles', include_zero=True),
        end_angle=cumsum('angles'),
        line_color="white",
        fill_color='colors',
        legend_field='fruits',
        source=source_fruit)
p2.legend.orientation = "horizontal"
p2.legend.location = "top"        
show(p2) 

### Sunspots Dataset

Sunspots are impermanent phenomena on the Sun's photosphere that appear darker than the surrounding areas. Sunspots occur in pairs of opposite magnetic polarities. Their number varies according to the **`11-year`** solar cycle.


Source: https://en.wikipedia.org/wiki/Sunspot

Database from SIDC - Solar Influences Data Analysis Center - the solar physics research department of the Royal Observatory of Belgium. SIDC website

In [10]:
df = pd.read_csv('Sunspots.csv', index_col=0)
df.head()

,Date,Monthly Mean Total Sunspot Number
0,1749-01-31,96.7
1,1749-02-28,104.3
2,1749-03-31,116.7
3,1749-04-30,92.8
4,1749-05-31,141.7


In [11]:
df.rename(columns = {'Monthly Mean Total Sunspot Number':'Sunspots'}, inplace=True)
df.head()

,Date,Sunspots
0,1749-01-31,96.7
1,1749-02-28,104.3
2,1749-03-31,116.7
3,1749-04-30,92.8
4,1749-05-31,141.7


In [12]:
df.dtypes

Date         object
Sunspots    float64
dtype: object

In [13]:
df.Date = pd.to_datetime(df.Date)
df.dtypes

Date        datetime64[ns]
Sunspots           float64
dtype: object

In [14]:
df['Year']  = df.Date.dt.strftime('%Y').astype('int') 
df['Month'] = df.Date.dt.strftime('%m').astype('int') 
df['Day']   = df.Date.dt.strftime('%d').astype('int') 
df.head() 

,Date,Sunspots,Year,Month,Day
0,1749-01-31,96.7,1749,1,31
1,1749-02-28,104.3,1749,2,28
2,1749-03-31,116.7,1749,3,31
3,1749-04-30,92.8,1749,4,30
4,1749-05-31,141.7,1749,5,31


Create a line plot of the sunspots vs. time

In [15]:
p3 = figure(title="Sunspots", x_axis_label='Date', y_axis_label='Sunspots',
            height=400, width=900)
p3.line(x=df.Date.dt.strftime('%Y'), y=df.Sunspots, line_width=2)
show(p3)

Let's add a hover tool to the graph.

In [16]:
source = ColumnDataSource(data=dict(
    x=df.Year,          # We need x and y for plotting the graph
    y=df.Sunspots,
    m=df.Month,         # We want to show month-day-year when hover the mouse over the graph
    d=df.Day)
)

Defining the `hover` tool

In [17]:
TOOLTIPS = [
    ("    Date = ", "@m-@d-@x"),
    ("Sunspots = ", "@y{0.0}"),
]

In [18]:
# Mouse over graph to visualize values
p4 = figure(title="Sunspots", 
            x_axis_label='Date', y_axis_label='Sunspots',
            tooltips=TOOLTIPS, height=400, width=900)
p4.line(line_width=2, source=source)
show(p4)

You can see the date and sunspot value for every point in the graph.

### Adding box annotations

In [19]:
from bokeh.models import BoxAnnotation

In [20]:
low_box  = BoxAnnotation(top=100,    fill_alpha=0.2, fill_color='palegreen')
high_box = BoxAnnotation(bottom=300, fill_alpha=0.2, fill_color='lightcoral')

In [21]:
p4.add_layout(low_box)
p4.add_layout(high_box)
show(p4)

## Linked Brushing

Brush over any graph and the point selection reflects in all charts.

### Automobile Dataset

We will use the Automobile Data Set [https://archive.ics.uci.edu/ml/datasets/automobile] from the UCI Machine Learning Repository [https://archive-beta.ics.uci.edu/]. It includes categorical and continuous variables. 

In [22]:
# Defining the headers
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration", "num_doors", 
           "body_style", "drive_wheels", "engine_location", "wheel_base", "length", "width", 
            "height", "curb_weight", "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm", "city_mpg", 
            "highway_mpg", "price"]

In [23]:
dfa = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data",
                  header=None, names=headers, na_values="?" )
dfa.head()

,symboling,normalized_losses,make,fuel_type,...,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,...,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,...,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,...,5000.0,19,26,16500.0
3,2,164.0,audi,gas,...,5500.0,24,30,13950.0
4,2,164.0,audi,gas,...,5500.0,18,22,17450.0


### Brush tooltip

We will start by creating a common column data source for all the plots.

In [24]:
sourceA = ColumnDataSource(data=dict(
    Price  = dfa.price, 
    Hmpg   = dfa.highway_mpg,
    Cmpg   = dfa.city_mpg,
    Hpower = dfa.horsepower)
    )

In [25]:
TOOLS = "box_select, lasso_select, reset, help"

Creating a plot for `Price` vs. `Hmpg`

In [26]:
# Creating a plot for Price vs Hmpg
f1 = figure(tools = TOOLS, y_range=[0, 50000], title='Car Price vs. Highway (mpg)')
f1.scatter('Hmpg', 'Price', source=sourceA, color='orangered', size=8);

Creating a plot for `Price` vs. `Cmpg`

In [27]:
# Creating a plot for Price vs Cmpg
f2 = figure(tools = TOOLS, y_range=[0, 50000], title='Car Price vs. City (mpg)')
f2.scatter('Cmpg', 'Price', source=sourceA, color='orange', size=8);

Creating a plot for `Price` vs. `Hpower`

In [28]:
# Creating a plot for Price vs Hpower
f3 = figure(tools = TOOLS, y_range=[0, 50000], title='Car Price vs. Horsepower')
f3.scatter('Hpower', 'Price', source=sourceA, color='steelblue', size=8);

Creating a grid

In [29]:
grid1 = gridplot([[f1, f2, f3]], width=300, height=300)
show(grid1)

Select a set of points in any graph, and see the automathic selection in the other two scatterplots.

Using the layout `row`

In [30]:
plot_r = row([f1, f2, f3])
show(plot_r)

In [31]:
# Creating another grid
grid2 = gridplot([[f1, f2], [f3]], width=350, height=300)
show(grid2)

In [32]:
grid3 = gridplot([[f1, f2], [None, f3]], width=350, height=300)
show(grid3)

In [33]:
grid4 = gridplot([[f1], [f2], [f3]], width=350, height=200)
show(grid4)

Using the layout `column`

In [34]:
plot_c = column([f1, f2, f3])
show(plot_c)

## Conclusions

- Interactive visualizations significantly enhance data exploration, allowing users to gain deeper insights.
- Bokeh's tools, like Hover Tool and linked brushing, facilitate detailed analysis and cross-visualization data comparisons.
- Implementing interactive features such as tooltips and selection highlights provides a richer, more engaging user experience.
- Bokeh can effectively handle different data visualizations, from time series to categorical comparisons, making it a versatile tool for data scientists.

## References

- https://bokeh.org/
- https://docs.bokeh.org/en/latest/docs/reference/layouts.html